In [14]:
##import libraries
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from mediapipe.python.solutions.drawing_utils import _normalized_to_pixel_coordinates as denormalize_coordinates

from filterpy.kalman import KalmanFilter
%matplotlib inline

In [15]:
# Meidapipe drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_facemesh = mp.solutions.face_mesh
denormalize_coordinates = mp_drawing._normalized_to_pixel_coordinates

In [16]:
#Eyes landmark points

all_left_eye_idxs = list(mp_facemesh.FACEMESH_LEFT_EYE)
# flatten and remove duplicates
all_left_eye_idxs = set(np.ravel(all_left_eye_idxs)) 
 
# Landmark points corresponding to right eye
all_right_eye_idxs = list(mp_facemesh.FACEMESH_RIGHT_EYE)
all_right_eye_idxs = set(np.ravel(all_right_eye_idxs))
 
# Combined for plotting - Landmark points for both eye
all_idxs = all_left_eye_idxs.union(all_right_eye_idxs)
 
# The chosen 12 points:   P1,  P2,  P3,  P4,  P5,  P6
chosen_left_eye_idxs  = [362, 385, 387, 263, 373, 380]
chosen_right_eye_idxs = [33,  160, 158, 133, 153, 144]
all_chosen_idxs = chosen_left_eye_idxs + chosen_right_eye_idxs

#             p1  p2  p3  p4  p5   p6   p7   p8
mouth_idxs = [61, 39, 0, 269, 291, 405, 17, 181]
mouth_idx_all = [61, 146, 146, 91, 91, 181, 181, 84, 84, 17, 17, 314, 314, 405, 405, 321,
321, 375, 375, 291, 61, 185, 185, 40, 40, 39, 39, 37, 37, 0, 0, 267, 267,
269, 269, 270, 270, 409, 409, 291, 78, 95, 95, 88, 88, 178, 178, 87, 87, 14,
14, 317, 317, 402, 402, 318, 318, 324, 324, 308, 78, 191, 191, 80, 80, 81,
81, 82, 82, 13, 13, 312, 312, 311, 311, 310, 310, 415, 415, 308,]

In [17]:
#distance calculation
def distance(point_1, point_2):
    """Calculate l2-norm between two points"""
    dist = sum([(i - j) ** 2 for i, j in zip(point_1, point_2)]) ** 0.5
    return dist

In [18]:
#Eye Aspect Ratio for one eye 
def get_ear(landmarks, refer_idxs, frame_width, frame_height):
    try:
        # Compute the euclidean distance between the horizontal
        coords_points = []
        for i in refer_idxs:
            lm = landmarks[i]
            coord = denormalize_coordinates(lm.x, lm.y, 
                                             frame_width, frame_height)
            coords_points.append(coord)
 
        # Eye landmark (x, y)-coordinates
        P2_P6 = distance(coords_points[1], coords_points[5])
        P3_P5 = distance(coords_points[2], coords_points[4])
        P1_P4 = distance(coords_points[0], coords_points[3])
 
        # Compute the eye aspect ratio
        ear = (P2_P6 + P3_P5) / (2.0 * P1_P4)
 
    except:
        ear = 0.0
        coords_points = None
 
    return ear, coords_points

In [19]:
#Avergae eye aspect ratio using both eyes
def calculate_avg_ear(landmarks, left_eye_idxs, right_eye_idxs, image_w, image_h):
    """Calculate Eye aspect ratio"""
 
    left_ear, left_lm_coordinates = get_ear(
                                      landmarks, 
                                      left_eye_idxs, 
                                      image_w, 
                                      image_h
                                    )

    right_ear, right_lm_coordinates = get_ear(
                                      landmarks, 
                                      right_eye_idxs, 
                                      image_w, 
                                      image_h
                                    )
                                    
    Avg_EAR = (left_ear + right_ear) / 2.0
 
    return Avg_EAR, (left_lm_coordinates, right_lm_coordinates)

In [20]:
# Mouth aspect ratio calculation
def get_mar(landmarks, refer_idxs, frame_width, frame_height):
    try:
        # Compute the euclidean distance between the horizontal
        coords_points = []
        for i in refer_idxs:
            lm = landmarks[i]
            coord = denormalize_coordinates(lm.x, lm.y, 
                                             frame_width, frame_height)
            coords_points.append(coord)
 
        #              p1  p2  p3  p4  p5   p6   p7   p8
        #mouth_idxs = [61, 39, 0, 269, 291, 405, 17, 181]
        #Mouth landmark (x, y)-coordinates
        P2_P8 = distance(coords_points[1], coords_points[7])
        P3_P7 = distance(coords_points[2], coords_points[6])
        P4_P6 = distance(coords_points[3], coords_points[5])
        P1_P5 = distance(coords_points[0], coords_points[4])
        # (p2-p8) +(p3-p7) +(p4-p6)/ 3(p1-p5)
        # mouth_idxs = [61, 39, 269,  291, 405, 181]
        # Compute the mouth aspect ratio
        mar = (P2_P8 + P3_P7 + P4_P6) / (3.0 * P1_P5)
 
    except:
        mar = 0.0
        coords_points = None
 
    return mar, coords_points

In [25]:

def detectDrowsiness(fileName, frameCount):

    images  = cv2.imread(fileName)[:, :, ::-1]
    EAR = 0
    MAR = 0

    for idx, image in enumerate([images]):
        
        image = np.ascontiguousarray(image)
        imgH, imgW, _ = image.shape
    
        # Creating a copy of the original image for plotting the EAR value
        custom_chosen_lmk_image = image.copy()
    
        # Running inference using static_image_mode
        with mp_facemesh.FaceMesh(refine_landmarks=True) as face_mesh:
            results = face_mesh.process(image).multi_face_landmarks
    
            # If detections are available.
            if results:
                for face_id, face_landmarks in enumerate(results):
                    landmarks = face_landmarks.landmark
                    EAR, _ = calculate_avg_ear(
                            landmarks, 
                            chosen_left_eye_idxs, 
                            chosen_right_eye_idxs, 
                            imgW, 
                            imgH
                        )
                    MAR, _ = get_mar(
                            landmarks, 
                            mouth_idxs,
                            imgW, 
                            imgH
                    )

                print(EAR)
                print(MAR)
                if (EAR > 0.2 and MAR < 0.6):
                    frameCount = 0
                elif (EAR < 0.2 or MAR > 0.6):
                    frameCount += 1
                
    return frameCount

In [26]:
fName = "test-open-eyes.jpg"
Count = 0
Count = detectDrowsiness(fName, Count)
drowsy = False

if (Count >= 1):
    drowsy = True
    print("Drowsy")
else: 
    drowsy = False
    print("Not Drowsy")

0.24472289283021406
0.42134970619608075
Not Drowsy


: 